# Before starting

- Copy the protein fasta file of the organism in the directory "/data/annotations/fasta/"
- Be sure you have the trained deep learning models to do the compartment prediction of the proteins in the folder "/bin/compartmentPredictions/deepModels". It can be downloaded from "http://doi.org/10.5281/zenodo.4436488"
- Install the eggNog annotation software: https://github.com/eggnogdb/eggnog-mapper
- Install the python libraries:Tensorflow, keras, biopython, pandas, numpy, cobrapy, Framed (https://github.com/cdanielmachado/framed)

# Compartment prediction of the proteins

In [15]:
import sys
sys.path.insert(0, 'bin/compartmentPrediction/')
from predict import predict

In [2]:
predict("data/annotations/fasta/example.fasta")

predicting
Predicted: 1002 proteins
(1001, 4)
Predicted: 2003 proteins
(1001, 4)
Predicted: 3004 proteins
(1001, 4)
Predicted: 4005 proteins
(1001, 4)
Predicted: 5006 proteins
(1001, 4)
(997, 4)


# Checking compartment prediction (Optional)

In [3]:
from Bio import SeqIO

In [18]:
import numpy as np
import pandas as pd

In [7]:
predictions = np.load("data/annotations/localization/example.fasta.npy")

In [26]:
organismPath="data/annotations/fasta/example.fasta"
i = 0
endo = []   
mito = []   
pero = []
other = []
ids = []
seqs = []
for record in SeqIO.parse(organismPath, "fasta"):
    pred=predictions[i]    
    endo.append(format(pred[0], ">-04.3f"))        
    mito.append(format(pred[1], ">-04.3f"))       
    pero.append(format(pred[2], ">-04.3f")) 
    other.append(format(pred[3], ">-04.3f")) 
    ids.append(record.id)
    seqs.append(str(record.seq))
    i =i+1
dataset = pd.DataFrame({'Id': ids, 'Sequences': seqs, 'Endoplasmic reticulum': endo, 'Mitochondria': mito,'Peroxisome': pero, 'Cytoplasm and others': other})
 

In [27]:
dataset

,Id,Sequences,Endoplasmic reticulum,Mitochondria,Peroxisome,Cytoplasm and others
0,NP_001018029.1,MRNELYQLWCVASAARGVAKSSFVRANSAMCEYVRTSNVLSRWTRD...,0.001,0.768,0.012,0.426
1,NP_001018030.1,MSIVYNKTPLLRQFFPGKASAQFFLKYECLQPSGSFKSRGIGNLIM...,0.000,1.000,0.002,0.001
2,NP_001018031.2,MGKCSMKKKGVGKNVGVGKKVQKKRSISTAERKRTKLQVEKLNKSS...,0.000,0.001,0.000,1.000
3,NP_001018032.1,MTVLIKLGLRILHVYKGFFRKVILKYFFFSSEHTKVNKKSSMHAFL...,0.018,0.640,0.009,0.225
4,NP_001018033.3,MSGYFNHLSSNAHFANIQADQGFIGDATGTSSDHGSSGMVDFALQL...,0.019,0.543,0.014,0.647
5,NP_001027023.1,MNSAGRVHRSRAGSRGHAAISPLTMASFSVARGIRSSNVYDDTDDE...,0.019,0.760,0.001,0.832
6,NP_001027534.1,MTLVGKLVHISIDLVLVSTCLAGIKRNTGLTPKLETLDNQTMRNYM...,0.006,0.434,0.008,0.558
7,NP_001032571.3,MLAMKSFSQVSKSYKASAPSKKLTTLFYVAYITLGLTTPFLLPARM...,0.050,0.786,0.004,0.046
8,NP_001032572.1,MSTAFRKIKLIFKKSDSQYPQNYRAEIKSRNKNTVITRHDLLIAHE...,0.000,0.174,0.007,0.774
9,NP_001032573.1,MQIKNIVAVLATVTAINAQVGIEPNATTPNATQPNATQPNTTLPTA...,0.018,0.340,0.002,0.657


# Functional annotation

download eggnog-mapper from https://github.com/eggnogdb/eggnog-mapper. 

In [4]:
import subprocess

In [7]:
python2_command= "bin/eggnog-mapper-master/emapper.py -i \"data/annotations/fasta/example.fasta\" --database euk --output \"data/annotations/functional/example.fasta\" -m diamond --override --cpu 32"

In [8]:
process = subprocess.Popen(python2_command.split(), stdout=subprocess.PIPE)
output, error = process.communicate() 

In [9]:
output

b'/usr/local/bin/diamond /mnt/home/scsandra/proj/reconstructionPipeline/bin/eggnog-mapper-master\n'

In [10]:
error

# Running CarveMe

In [1]:
import sys
sys.path.insert(0, 'bin/')
import CreateModelEggNogPool 

In [2]:
import logging
serf_logger = logging.getLogger('cobra')
serf_logger.setLevel(logging.ERROR)

In [3]:
CreateModelEggNogPool.carveme_pipeline("Scer", "Saccharomyces_cerevisiae_S288C_GCF_000146045.2","data/annotations/functional/example.fasta.emapper.annotations","data/annotations/fasta/example.fasta","data/annotations/localization/example.fasta.npy")

Scores calculated
Solution 0
Objective: 831.1412724138487
Status: Unknown

Objective: 1.0501703521544485
Status: Optimal

Solution 1
Objective: 831.1412724137851
Status: Unknown

Objective: 1.039036251071051
Status: Optimal

Solution 2
Objective: 831.1412724137297
Status: Unknown

Objective: 0.9694150580028384
Status: Optimal

Solution 3
Objective: 831.1412724136583
Status: Unknown

Objective: 0.885741661129646
Status: Optimal

Solution 4
Objective: 831.1412724136544
Status: Unknown

Objective: 1.0501703521544765
Status: Optimal

Solution 0
Objective: 918.752519678168
Status: Unknown

Objective: 8.66130847189272
Status: Optimal

Solution 1
Objective: 918.7525196781347
Status: Unknown

Objective: 8.661308471892717
Status: Optimal

Solution 2
Objective: 918.7525196781282
Status: Unknown

Objective: 8.480229065144844
Status: Optimal

Solution 3
Objective: 917.0057090763559
Status: Unknown

Objective: 8.702477764124382
Status: Optimal

Solution 4
Objective: 917.0057090763547
Status: Unknow

The resulting model contains two numbers corresponding to the objective result of the two MILP problems (models with open and closed bounds)

Remember to create few models and selecte the one with bigger objective results. 

# Analyzing the resulting model

In [1]:
import cobra

In [6]:
model = cobra.io.read_sbml_model("results/Scer-831.1412724137351-918.0537954374283.sbml")

#### Fixing the BIOMASS reaction

In [7]:
from cobra import Model, Reaction, Metabolite
reaction = Reaction('BIOMASS')
reaction.name = model.reactions.BIOMASS.name
reaction.add_metabolites({
    model.metabolites.get_by_id("13BDglcn_1g_c"): -0.253,
    model.metabolites.ash_1g_c: -0.048,
    model.metabolites.atp_c: -60.0,
    model.metabolites.chitin_1g_c: -0.01,
    model.metabolites.dna_1g_c: -0.005,
    model.metabolites.glycogen_1g_c: -0.1,
    model.metabolites.h2o_c: -60.0,
    model.metabolites.mannan_1g_c: -0.07,
    model.metabolites.phospholipid_1g_c: -0.04,
    model.metabolites.protein_1g_c: -0.4,
    model.metabolites.rna_1g_c: -0.058,
    model.metabolites.sterol_1g_r: -0.001,
    model.metabolites.trehalose_1g_c: -0.015,
    model.metabolites.ficytc_c: -0.0001,
    model.metabolites.focytc_c: -0.0001,
    model.metabolites.retn_c: -0.0001,
    model.metabolites.thmtp_c: -0.0001,
    model.metabolites.ribflv_c: -0.0001,
    model.metabolites.btn_c: -0.0001,
    model.metabolites.q6_m: -0.0001,
    model.metabolites.coa_c: -0.0001,
    model.metabolites.thf_c: -0.0001,
    model.metabolites.adp_c: 60.0,
    model.metabolites.biomass_1g_c: 1.0,
    model.metabolites.h_c: 60.0,
    model.metabolites.pi_c: 60.0
})

model.reactions.BIOMASS.remove_from_model()
model.add_reaction(reaction)

In [8]:
model.objective= "BIOMASS"
model.optimize()

,fluxes,reduced_costs
UF00005_C,0.000000,0.000000e+00
UF00014_C,0.000000,0.000000e+00
UF00015_C,0.000000,-3.019926e-03
UF00016_C,0.000000,0.000000e+00
UF00017_C,0.000000,-3.019926e-03
...,...,...
UF03498_E,-0.000000,0.000000e+00
UF02829_E,0.000000,-1.117372e-01
UF03138_E,0.000000,-2.610079e-01
UF00779_x,0.000000,0.000000e+00


In [9]:
cobra.io.write_sbml_model(model,"results/Scer-831.1412724137351-918.0537954374283.sbml")

### Analyze individual models in the assembly

In [10]:
def extract_single_model_from_ensemble(position, model):
    newModel = model.copy()
    reactiond = {}
    for reaction in model.reactions:
        name = reaction.name.split('_')       
        ensemble=[]
        for n in name[1:]:
            try:
                n = int(n)
                ensemble.append(n) 
            except:
                None
        reactiond[reaction.id]=ensemble       
    
    for reaction in model.reactions:
        presence=reactiond[reaction.id]          
        if presence[position]==0:           
            newModel.remove_reactions([reaction.id])
            
    return newModel 
def get_compartments_rxn(reaction):
    comp =[]
    for metabolite in reaction.metabolites:
        comp.append(metabolite.compartment)
    return list(set(comp))

def get_compartments(model):
    cyt=0
    nucleus=0
    ret=0
    mit=0
    per=0
    for reaction in model.reactions:
        compartments= get_compartments_rxn(reaction)
        for c in compartments:
            if 'c' in c:
                cyt = cyt+1
            elif 'r' in c:
                ret = ret+1
            elif 'x' in c:
                per=per+1
            elif 'n' in c:
                nucleus=nucleus+1
            elif 'm' in c:
                mit =mit+1
    return [cyt, nucleus, ret, mit, per]

In [11]:
for i in range(25):
    newModel = extract_single_model_from_ensemble(i, model)
    print("Model " +str(i))
    print("Growth: " + str(newModel.optimize().objective_value))
    print("Number of reactions: " + str(len(newModel.reactions)))
    print("Number of compounds: " + str(len(newModel.metabolites)))
    [cyt, nucleus, ret, mit, per]=get_compartments(newModel)
    print("Reactions in cytoplasm: " + str(cyt))
    print("Reactions in endoplasmic reticulum: " + str(ret))
    print("Reactions in mitochondria: " + str(mit))
    print("Reactions in peroxisome: " + str(per))
    print("--------------------------------")

Model 0
Growth: 0.9562314807679013
Number of reactions: 1999
Number of compounds: 1571
Reactions in cytoplasm: 1172
Reactions in endoplasmic reticulum: 86
Reactions in mitochondria: 341
Reactions in peroxisome: 329
--------------------------------
Model 1
Growth: 0.9562314807678831
Number of reactions: 2004
Number of compounds: 1571
Reactions in cytoplasm: 1174
Reactions in endoplasmic reticulum: 89
Reactions in mitochondria: 341
Reactions in peroxisome: 329
--------------------------------
Model 2
Growth: 0.9562314807678772
Number of reactions: 2001
Number of compounds: 1571
Reactions in cytoplasm: 1173
Reactions in endoplasmic reticulum: 86
Reactions in mitochondria: 341
Reactions in peroxisome: 329
--------------------------------
Model 3
Growth: 0.9575360247920888
Number of reactions: 2006
Number of compounds: 1571
Reactions in cytoplasm: 1177
Reactions in endoplasmic reticulum: 89
Reactions in mitochondria: 347
Reactions in peroxisome: 312
--------------------------------
Model 4
